In [ ]:
!pip install requests beautifulsoup4

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Load the Excel file containing the URLs (no header)
excel_file_path = '/content/drive/MyDrive/Text_Analytics/Project/TA Project links.xlsx'
output_dir = "/content/drive/MyDrive/AAAI_Papers"  # Directory to save the PDFs
os.makedirs(output_dir, exist_ok=True)

# Custom headers to simulate a browser
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/114.0.0.0 Safari/537.36"
    )
}

# Function to download a PDF
def download_pdf(pdf_url, save_path):
    try:
        response = requests.get(pdf_url, headers=headers, stream=True)
        if response.status_code == 200 and response.headers.get('Content-Type') == 'application/pdf':
            with open(save_path, "wb") as file:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        file.write(chunk)
            print(f"Downloaded: {save_path}")
        else:
            print(f"Invalid PDF or Failed to download: {pdf_url} - Status Code: {response.status_code}")
    except Exception as e:
        print(f"Error downloading {pdf_url}: {e}")

# Function to scrape PDF links from a URL
def scrape_pdfs_from_url(url):
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            pdf_links = []

            # Find all <a> tags with "href"
            for link in soup.find_all("a", href=True):
                href = link["href"]
                # Check if the link ends with ".pdf" or matches the "/article/view/" pattern
                if href.endswith(".pdf") or "/article/view/" in href:
                    if not href.startswith("http"):
                        href = f"https://ojs.aaai.org{href}"
                    if not href.endswith(".pdf"):
                        href = href.replace("/view/", "/download/") + ".pdf"  # Adjust to point to PDFs
                    pdf_links.append(href)

            return pdf_links
        else:
            print(f"Failed to fetch the webpage: {url}")
            return []
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return []

# Load URLs from the Excel file (second column, no header)
df = pd.read_excel(excel_file_path, header=None)
urls = df.iloc[:, 1]  # Second column (index 1)

# Loop through the URLs and scrape PDFs
for url in urls:
    print(f"Scraping URL: {url}")
    pdf_links = scrape_pdfs_from_url(url)
    for pdf_url in pdf_links:
        pdf_name = pdf_url.split("/")[-1]
        save_path = os.path.join(output_dir, pdf_name)

        # Ensure directory exists
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        download_pdf(pdf_url, save_path)


Streaming output truncated to the last 5000 lines.
Downloaded: /content/drive/MyDrive/AAAI_Papers/28867.pdf
Invalid PDF or Failed to download: https://ojs.aaai.org/index.php/AAAI/article/download/28445/28868.pdf - Status Code: 200
Invalid PDF or Failed to download: https://ojs.aaai.org/index.php/AAAI/article/download/28446.pdf - Status Code: 404
Downloaded: /content/drive/MyDrive/AAAI_Papers/28869.pdf
Invalid PDF or Failed to download: https://ojs.aaai.org/index.php/AAAI/article/download/28446/28870.pdf - Status Code: 200
Invalid PDF or Failed to download: https://ojs.aaai.org/index.php/AAAI/article/download/28447.pdf - Status Code: 404
Downloaded: /content/drive/MyDrive/AAAI_Papers/28871.pdf
Invalid PDF or Failed to download: https://ojs.aaai.org/index.php/AAAI/article/download/28447/28872.pdf - Status Code: 200
Invalid PDF or Failed to download: https://ojs.aaai.org/index.php/AAAI/article/download/28448.pdf - Status Code: 404
Downloaded: /content/drive/MyDrive/AAAI_Papers/28873.pdf
I